In [1]:
# predict the next char
import tensorflow as tf
import numpy as np
import random
import pandas as pd
import re
%matplotlib inline

In [2]:
def make_mini_batch(train_data, size_of_mini_batch, length_of_sequences):
    inputs  = ''
    outputs = ''
    for _ in range(size_of_mini_batch):
        index   = random.randint(0, len(train_data) - length_of_sequences - 1)
        inputs  += train_data[index:index + length_of_sequences]
        outputs += train_data[index + length_of_sequences]
    inputs = np.eye(num_of_char)[[ord(x) for x in inputs if ord(x) < 128]]
    outputs = np.eye(num_of_char)[[ord(x) for x in outputs if ord(x) < 128]]
    inputs  = inputs.reshape(-1, length_of_sequences, 128)
    outputs = outputs.reshape(-1, 128)
    return (inputs, outputs)

In [3]:
def make_prediction_initial(train_data, index, length_of_sequences):
    inputs = train_data[index:index + length_of_sequences]
    return np.eye(num_of_char)[[ord(x) for x in inputs if ord(x) < 128]]    
#inputs, _  = make_prediction_initial(train_data, 0, length_of_initial_sequences)
#inputs.shape

In [21]:
train_data_path             = "./input/stack.tex"
num_of_char                 = 128
num_of_input_nodes          = 128
num_of_hidden_nodes         = 512
num_of_output_nodes         = 128
length_of_sequences         = 50
num_of_training_epochs      = 14000
length_of_initial_sequences = 50
num_of_prediction_epochs    = 1000
size_of_mini_batch          = 20
learning_rate               = 0.0005
forget_bias                 = 1.0

In [5]:
#train_data = np.load(train_data_path)
f = open(train_data_path)
text = f.read()
f.close()
train_data = ''.join([x for x in text if ord(x) < 128])

In [6]:
def inference(input_ph, size_of_mini_batch_ph, lstm_output_keep_prob):
    with tf.name_scope("inference") as scope:
        weight1_var = tf.Variable(tf.truncated_normal([num_of_input_nodes, num_of_hidden_nodes], stddev=0.1), name="weight1")
        weight2_var = tf.Variable(tf.truncated_normal([num_of_hidden_nodes, num_of_output_nodes], stddev=0.1), name="weight2")
        bias1_var   = tf.Variable(tf.truncated_normal([num_of_hidden_nodes], stddev=0.1), name="bias1")
        bias2_var   = tf.Variable(tf.truncated_normal([num_of_output_nodes], stddev=0.1), name="bias2")

        weight1_hist = tf.histogram_summary("layer1/weights", weight1_var)
        weight2_hist = tf.histogram_summary("layer2/weights", weight2_var)
        bias1_hist = tf.histogram_summary("layer1/biases", bias1_var)
        bias2_hist = tf.histogram_summary("layer2/biases", bias2_var)
        
        # pre rnn
        in1 = tf.transpose(input_ph, [1, 0, 2])         # (batch, sequence, data) -> (sequence, batch, data)
        in2 = tf.reshape(in1, [-1, num_of_input_nodes]) # (sequence, batch, data) -> (sequence * batch, data)
        in3 = tf.matmul(in2, weight1_var) + bias1_var
        in4 = tf.split(0, length_of_sequences, in3)     # sequence * (batch, data)

        cell = tf.nn.rnn_cell.BasicLSTMCell(num_of_hidden_nodes, forget_bias=forget_bias, state_is_tuple=True)
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=lstm_output_keep_prob)
        cell = tf.nn.rnn_cell.MultiRNNCell([cell] * 2, state_is_tuple=True)
        initial_state = cell.zero_state(size_of_mini_batch_ph, tf.float32)
        rnn_output, states_op = tf.nn.rnn(cell, in4, initial_state=initial_state, dtype=tf.float32)

        output_op = tf.matmul(rnn_output[-1], weight2_var) + bias2_var
    return output_op, rnn_output

In [7]:
def loss(output_op):
    with tf.name_scope("loss") as scope:
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output_op, supervisor_ph))
        loss_op = cross_entropy
        tf.scalar_summary("loss", loss_op)
    return loss_op

In [8]:
def train(loss_op):
    with tf.name_scope("training") as scope:
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
        #optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(loss_op)
    return training_op

In [22]:
with tf.Graph().as_default():
    input_ph      = tf.placeholder(tf.float32, [None, length_of_sequences, num_of_input_nodes], name="input")
    supervisor_ph = tf.placeholder(tf.float32, [None, num_of_output_nodes], name="supervisor")
    size_of_mini_batch_ph = tf.placeholder(tf.int32, name="size_of_mini_batch")
    lstm_output_keep_prob_ph = tf.placeholder(tf.float32, name="lstm_output_keep_prob")

    output_op, rnn_output = inference(input_ph, size_of_mini_batch_ph, lstm_output_keep_prob_ph)
    loss_op = loss(output_op)
    training_op = train(loss_op)

    summary_op = tf.merge_all_summaries()

    with tf.Session() as sess:
        summary_writer = tf.train.SummaryWriter("data", graph=sess.graph)
        """
        # random seed fix
        random.seed(0)
        np.random.seed(0)
        tf.set_random_seed(0)
        """
        epoch_start = 0
        ckpt = tf.train.get_checkpoint_state("./train/")
        saver = tf.train.Saver()
        if ckpt and tf.gfile.Exists(ckpt.model_checkpoint_path):
            print("Reading model parameters from %s" % ckpt.model_checkpoint_path)
            saver.restore(sess, ckpt.model_checkpoint_path)
            epoch_start = int(re.search(r"(\d+)$", ckpt.model_checkpoint_path).group())
        else:
            print("Created model with fresh parameters.")
            sess.run(tf.initialize_all_variables())

        for epoch in range(epoch_start, num_of_training_epochs):
            inputs, supervisors = make_mini_batch(train_data, size_of_mini_batch, length_of_sequences)

            train_dict = {
                input_ph:      inputs,
                supervisor_ph: supervisors,
                size_of_mini_batch_ph: size_of_mini_batch,
                lstm_output_keep_prob_ph: 0.5,
            }
            rnnout, _ = sess.run([rnn_output, training_op], feed_dict=train_dict)

            if epoch % 10 == 0:
                summary_str, train_loss = sess.run([summary_op, loss_op], feed_dict=train_dict)
                summary_writer.add_summary(summary_str, epoch)
                print("train#%d, train loss: %e" % (epoch, train_loss))

            if epoch % 10 == 0:
                saver.save(sess, "train/translate.ckpt", global_step=epoch)

        inputs = make_prediction_initial(train_data, 0, length_of_initial_sequences)
        outputs = np.array([[0 for _ in range(128)]])

        print("prediction: ")
        for epoch in range(num_of_prediction_epochs):
            pred_dict = {
                input_ph:  [inputs],
                size_of_mini_batch_ph: 1,
                lstm_output_keep_prob_ph: 1.0
            }
            output = sess.run(output_op, feed_dict=pred_dict)
            output_onehotvec = np.eye(num_of_char)[[np.argmax(output)]]
            inputs  = np.delete(inputs, 0, 0)
            inputs  = np.append(inputs, output_onehotvec, 0)
            outputs = np.append(outputs, output_onehotvec, 0)
        
        outputs  = np.delete(outputs, 0, 0)
        output_ascii = np.argmax(outputs.reshape(num_of_prediction_epochs, num_of_output_nodes), axis=1)
        print("outputs:", ''.join([chr(x) for x in output_ascii]))

Reading model parameters from ./train/translate.ckpt-10990
train#10990, train loss: 1.163687e+00
train#11000, train loss: 9.561481e-01
train#11010, train loss: 8.963711e-01
train#11020, train loss: 1.968876e+00
train#11030, train loss: 8.193018e-01
train#11040, train loss: 9.574386e-01
train#11050, train loss: 1.119974e+00
train#11060, train loss: 1.226853e+00
train#11070, train loss: 7.975789e-01
train#11080, train loss: 1.182474e+00
train#11090, train loss: 6.701686e-01
train#11100, train loss: 6.690273e-01
train#11110, train loss: 8.425705e-01
train#11120, train loss: 4.172142e-01
train#11130, train loss: 1.294003e+00
train#11140, train loss: 6.723601e-01
train#11150, train loss: 6.293373e-01
train#11160, train loss: 1.551203e+00
train#11170, train loss: 1.132574e+00
train#11180, train loss: 8.830321e-01
train#11190, train loss: 3.862577e-01
train#11200, train loss: 7.655944e-01
train#11210, train loss: 1.494378e+00
train#11220, train loss: 4.885359e-01
train#11230, train loss: 1.20